
### The data path in the file is for running on local machine
### Same path were changed when Notebook was run on kaggle
### To run file will need to change path accordingly





In [1]:
import numpy as np # linear algebra
import pandas as pd 
from numpy import mean
from numpy import std
from numpy import arange

from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, make_scorer
# from sklearn.preprocessing import Imputer
# from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

# Custom scorer for cross validation
scorer = make_scorer(f1_score, greater_is_better=True, average = 'macro')

from sklearn.datasets import make_classification

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, make_scorer
# from sklearn.preprocessing import Imputer
# from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

# Custom scorer for cross validation
scorer = make_scorer(f1_score, greater_is_better=True, average = 'macro')

from sklearn.datasets import make_classification

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_set = pd.read_csv('../content/drive/My Drive/Machine Learning Course Project/data/train_median_with_cols.csv')
test_set = pd.read_csv('../content/drive/My Drive/Machine Learning Course Project/data/test_median_with_cols.csv')

In [4]:
# Shape
print('Shape of train: ', train_set.shape)
print('Shape of test: ', test_set.shape)

# Checking if there are any missing values 
print('Number of NaN values in train:', len(train_set.columns[train_set.isna().any()].tolist()))
print('Number of NaN values in test: ',len(test_set.columns[test_set.isna().any()].tolist()))

Shape of train:  (2973, 231)
Shape of test:  (7334, 231)
Number of NaN values in train: 0
Number of NaN values in test:  1


##Deciding Hyperparameters:
```
criterion : gini 
max_features : sqrt*(no_features) 
max_depth : none 
```




##Model Training

In [ ]:
scorer = make_scorer(f1_score, greater_is_better=True, average = 'macro')

In [7]:
def model_training(train,test):
    pipeline = Pipeline([('imputer', SimpleImputer(strategy = 'median')), 
                          ('scaler', MinMaxScaler())])
    y = train['Target']
    train.drop(columns=['Target'], inplace=True)

    # Fit and transform training data
    train_set = pipeline.fit_transform(train)
    test_set = pipeline.transform(test)

    model = DecisionTreeClassifier(criterion='gini', max_features='sqrt', max_depth=None, random_state=10, )


    predicts_result = []

    kf = KFold(n_splits=10)

    predicts_result = []
    for train_indices, test_indices  in kf.split(train_set):
        model.fit(train_set[train_indices], y[train_indices])
        predicts_result.append(model.predict(test_set))

    return predicts_result

In [8]:
# Removing certain columns not required for traning/predictions
train = train_set[train_set['Target'].notnull()].drop(columns = ['Id', 'idhogar', 'Unnamed: 0']).copy()
test = test_set[test_set['Target'].isnull()].drop(columns = ['Id', 'idhogar', 'Target', 'Unnamed: 0']).copy()

In [9]:
predictions = model_training(train,test)

In [11]:
results = pd.read_csv('../content/drive/My Drive/Machine Learning Course Project/data/test_median_with_cols.csv')
results['Target'] = np.array(predictions).mean(axis=0).round().astype(int)
results = results[['idhogar','Target']].copy()
test = pd.read_csv('../content/drive/My Drive/Machine Learning Course Project/data/test.csv')
test = test[['Id','idhogar']].copy()
submission = pd.merge(test,results, on='idhogar', how='outer')
submission.fillna(4, inplace=True)
submission.drop(columns='idhogar', inplace= True)
submission = submission.astype({'Target': int})
submission.to_csv('submission.csv', index = False)